In [1]:
import json
from langchain.schema import Document

def load_jsonl_as_documents(path):
    docs = []
    with open(path, 'r') as file:
        for line in file:
            data = json.loads(line)
            text = f"Question: {data.get("Question")} Answer: {data.get("Final answer")}"
            metadata = {k: v for k, v in data.items() if k in ["task_id"]}
            docs.append(Document(page_content=text, metadata=metadata))
    return docs

documents = load_jsonl_as_documents("metadata.jsonl")


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\austi\AppData\Local\Temp\ipykernel_944\3745702769.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\austi\anaconda3\envs\master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embedding_model)
vector_store.save_local("faiss_index")


In [ ]:
def retriever_node(state: MessagesState):
    """Simulates a LangGraph node that injects similar question context."""
    query = state["messages"][0].content
    retriever = vector_store.as_retriever(search_kwargs={"k": 1})
    results = retriever.get_relevant_documents(query)

    example_msg = HumanMessage(
        content=f"Here is a similar contract or question:\n\n{results[0].page_content}"
    )

    return {
        "messages": [sys_msg] + state["messages"] + [example_msg]
    }

In [11]:
from langgraph.graph import StateGraph, MessagesState
from langchain_core.messages import SystemMessage, HumanMessage
sys_msg = SystemMessage(content="You are a helpful assistant.")
graph = StateGraph(MessagesState)
graph.add_node("retriever", retriever_node)
graph.set_entry_point("retriever")
graph.set_finish_point("retriever")

rag_app = graph.compile()

output = rag_app.invoke({
    "messages": [HumanMessage(content="What is the first name of the only Malko Competition recipient from the 20th Century (after 1977) whose nationality on record is a country that no longer exists?")]
})

print(output["messages"][-1].content)


Here is a similar contract or question:

Question: What is the first name of the only Malko Competition recipient from the 20th Century (after 1977) whose nationality on record is a country that no longer exists? Answer: Claus
